In [1]:
import os
import torch
from transformers import (
        get_linear_schedule_with_warmup,
        BertTokenizer,
        AdamW,
        AutoModelForSequenceClassification,
        BertForSequenceClassification,
        BertModel,
        AutoConfig
        )
from torch.utils.data import DataLoader,dataset
import time
import numpy as np
from sklearn import metrics
from datetime import timedelta

In [2]:
data_dir='data'
def read_file(path):
    with open(path, 'r', encoding="UTF-8") as file:
        docus = file.readlines()
        newDocus = []
        for data in docus:
            newDocus.append(data)
    return newDocus
#建立数据集 
class Label_Dataset(dataset.Dataset):
    def __init__(self,data):
        self.data = data
    def __len__(self):#返回数据长度
        return len(self.data)
    def __getitem__(self,ind):
        onetext = self.data[ind]
        label, content = onetext.split('\t',1)
        label = torch.LongTensor([int(float(label))])
        return content,label
def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


In [3]:
trainContent = read_file(os.path.join(data_dir, "train1.csv")) 
testContent = read_file(os.path.join(data_dir, "test1.csv"))
valContent = read_file(os.path.join(data_dir, "val1.csv"))

traindataset =Label_Dataset( trainContent )
testdataset =Label_Dataset( testContent )
valdataset =Label_Dataset( valContent )

testdataloder = DataLoader(testdataset, batch_size=5, shuffle = False)
valdataloder = DataLoader(valdataset, batch_size=5, shuffle = False)
batch_size = 5
traindataloder = DataLoader(traindataset, batch_size=batch_size, shuffle = True)

class_list = [x.strip() for x in open(
        os.path.join(data_dir, "class.txt"),encoding='utf-8').readlines()]

In [4]:
pretrained_weights = 'bert/bert-base-chinese'#建立模型
tokenizer = BertTokenizer.from_pretrained(pretrained_weights,padding=True,truncation=True,max_length=512)
config = AutoConfig.from_pretrained(pretrained_weights ,num_labels=2) 
#单独指定config，在config中指定分类个数
nlp_classif = AutoModelForSequenceClassification.from_pretrained(pretrained_weights ,config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
nlp_classif = nlp_classif.to(device)
for p in nlp_classif.parameters():
        p.requires_grad=True
time_start = time.time() #开始时间
epochs = 3
gradient_accumulation_steps = 1
max_grad_norm =0.1  #梯度剪辑的阀值
require_improvement = 1000      # 若超过1000batch效果还没提升，则提前结束训练
savedir = './myfinetun-bert_chinese'
os.makedirs(savedir, exist_ok=True)

Some weights of the model checkpoint at bert/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [5]:
def train( model, traindataloder, testdataloder):
    start_time = time.time()
    model.train()
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, eps=1e-8)

    
    scheduler = get_linear_schedule_with_warmup(optimizer,
                num_warmup_steps=0, num_training_steps=len(traindataloder) * epochs)


    total_batch = 0  # 记录进行到多少batch
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    flag = False  # 记录是否很久没有效果提升
    
    for epoch in range(epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, epochs))
        for i, (sku_name, labels) in enumerate(traindataloder):
            model.train()
            
            ids = tokenizer.batch_encode_plus( sku_name,padding=True,truncation=True,max_length=512,
                #模型的配置文件中就是512，当有超过这个长度的会报错
                return_tensors='pt')#没有return_tensors会返回list！！！！
               
            labels = labels.squeeze().to(device) 
            outputs = model(ids["input_ids"].to(device), labels=labels,
                            attention_mask =ids["attention_mask"].to(device)  )
            
            loss, logits = outputs[:2]
            
            if gradient_accumulation_steps > 1:
                loss = loss / gradient_accumulation_steps
            
            loss.backward()
            
            if (i + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            
            if total_batch % 100 == 0:
                # 每多少轮输出在训练集和验证集上的效果
                truelabel = labels.data.cpu()
                predic = torch.argmax(logits,axis=1).data.cpu()
#                predic = torch.max(outputs.data, 1)[1].cpu()
                train_acc = metrics.accuracy_score(truelabel, predic)
                print(train_acc)
                dev_acc, dev_loss = evaluate( model, testdataloder)
                if dev_loss < dev_best_loss:
                    dev_best_loss = dev_loss
                    model.save_pretrained(savedir)                    
                    improve = '*'
                    last_improve = total_batch
                else:
                    improve = ''
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {1:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.2%},  Time: {5} {6}'
                print(msg.format(total_batch, loss.item(), train_acc, dev_loss, dev_acc, time_dif, improve))
                model.train()
            total_batch += 1
            if total_batch - last_improve > require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break
        if flag:
            break

def evaluate(model, testdataloder):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for sku_name, labels in testdataloder:
            ids = tokenizer.batch_encode_plus( sku_name,padding=True,truncation=True,max_length=512,return_tensors='pt')#没有return_tensors会返回list！！！！
               
            labels = labels.squeeze().to(device) 
            outputs = model(ids["input_ids"].to(device), labels=labels,attention_mask =ids["attention_mask"].to(device) )
            
            loss, logits = outputs[:2]
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predic = torch.argmax(logits,axis=1).data.cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)
    acc = metrics.accuracy_score(labels_all, predict_all)
    return acc, loss_total / len(testdataloder)

In [6]:
train(nlp_classif, traindataloder, valdataloder)    

e:\python\transformers\src\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch [1/3]
0.6


KeyboardInterrupt: 

In [10]:
#微调后测试集准确率
config = AutoConfig.from_pretrained(savedir,num_labels=2) 
nlp_classif2 = AutoModelForSequenceClassification.from_pretrained(savedir,config=config).to(device)
testContent = read_file(os.path.join(data_dir, "test1.csv"))
testdataset =Label_Dataset( testContent )
testdataloder = DataLoader(testdataset, batch_size=1, shuffle = False)
loss_total = 0
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
logits_all=[]
with torch.no_grad():
    for sku_name, labels in testdataloder:
        ids = tokenizer.batch_encode_plus( sku_name,truncation=True,
            max_length=512,  #模型的配置文件中就是512，当有超过这个长度的会报错
            padding=True,return_tensors='pt')#没有return_tensors会返回list！！！！

        labels = labels.squeeze().to(device) 
        outputs = nlp_classif2(ids["input_ids"].to(device), labels=labels,attention_mask =ids["attention_mask"].to(device) )
        loss, logits = outputs[:2]
        logits_all.append(logits.data.cpu().numpy())
        loss_total += loss
        labels = labels.data.cpu().numpy()
        predic = torch.argmax(logits,axis=1).data.cpu().numpy()
        labels_all = np.append(labels_all, labels)
        predict_all = np.append(predict_all, predic)
acc = metrics.accuracy_score(labels_all,predict_all)

In [12]:
predict_all.shape

(14000,)

In [7]:
acc

0.9492142857142857

In [13]:
with open("test_result.txt",'w') as f:
    for i in predict_all:
        f.write(str(i)+' ')


In [20]:
test_predict_all

array([], dtype=int32)

In [13]:
#微调后训练集准确率
trainContent = read_file(os.path.join(data_dir, "train1.csv"))
traindataset =Label_Dataset( trainContent )
traindataloder = DataLoader(traindataset, batch_size=1, shuffle = False)
loss_total = 0
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
logits_all=[]
with torch.no_grad():
    for sku_name, labels in traindataloder:
        ids = tokenizer.batch_encode_plus( sku_name,truncation=True,
            max_length=512,  #模型的配置文件中就是512，当有超过这个长度的会报错
            padding=True,return_tensors='pt')#没有return_tensors会返回list！！！！
        labels = labels.squeeze().to(device) 
        outputs = nlp_classif2(ids["input_ids"].to(device), labels=labels,attention_mask =ids["attention_mask"].to(device) )
        loss, logits = outputs[:2]
        logits_all.append(logits.data.cpu().numpy())
        loss_total += loss
        labels = labels.data.cpu().numpy()
        predic = torch.argmax(logits,axis=1).data.cpu().numpy()
        labels_all = np.append(labels_all, labels)
        predict_all = np.append(predict_all, predic)
acc = metrics.accuracy_score(labels_all, predict_all)

In [14]:
acc

0.9530982142857143

In [15]:
#模型检验
with torch.no_grad():
    inputs = tokenizer("生意超级好，但服务超级差劲哦！！！ 我们坐下后，没人擦桌子，没人上餐具的一刚！！！！ 后来更离谱来！！！ 虾饺，没醋的， 红豆冰，没勺子的！ 色拉，就一盘菜，没色拉酱的！！！ 什么和什么麻！！！ 怎么会这样的！！！！！ 差到家哦！！！！！！！！", return_tensors="pt").to(device)
    labels = torch.tensor([1]).unsqueeze(0).to(device)  # Batch size 1
    outputs = nlp_classif2(**inputs, labels=labels)
    loss = outputs.loss
    logits = outputs.logits
logits

In [14]:
testContent = read_file(os.path.join(data_dir, "test1.csv"))
testdataset =Label_Dataset( testContent )
testdataloder = DataLoader(testdataset, batch_size=1, shuffle = False)
nlp_classif.to(device)
nlp_classif.eval()
loss_total = 0
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
loss_total = 0
predict_all = np.array([], dtype=int)
labels_all = np.array([], dtype=int)
logits_all=[]
with torch.no_grad():
    for sku_name, labels in testdataloder:
        ids = tokenizer.batch_encode_plus( sku_name,truncation=True,
            max_length=512,  #模型的配置文件中就是512，当有超过这个长度的会报错
            padding=True,return_tensors='pt')#没有return_tensors会返回list！！！！

        labels = labels.squeeze().to(device) 
        outputs = nlp_classif(ids["input_ids"].to(device), labels=labels,attention_mask =ids["attention_mask"].to(device) )
        loss, logits = outputs[:2]
        logits_all.append(logits.data.cpu().numpy())
        loss_total += loss
        labels = labels.data.cpu().numpy()
        predic = torch.argmax(logits,axis=1).data.cpu().numpy()
        labels_all = np.append(labels_all, labels)
        predict_all = np.append(predict_all, predic)
acc = metrics.accuracy_score(labels_all, predict_all)